In [14]:
import pandas as pd
import numpy as np
from ANNbuilder import build_network
from pso import pso_optimization
from loss import mse_loss, sparse_categorical_crossentropy
from sklearn.metrics import classification_report

In [15]:
data = pd.read_csv(
    "/Users/ilya/Desktop/GitHub_Repositories/HW_University/Data_Mining/datasets/Iris.csv")
data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [16]:
data = data.drop(['Id'], axis=1)

In [17]:
data.shape

(150, 5)

In [18]:
data["Species"].value_counts()

Species
Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: count, dtype: int64

In [19]:
X = data.drop("Species", axis=1).values
y = data["Species"]

In [20]:
X[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [21]:
y.shape

(150,)

In [22]:
# Data separation and scaling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [23]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

# Converting class labels for training data
# 
y_train_encoded = encoder.fit_transform(y_train)

# Converting class labels for test data
y_test_encoded = encoder.transform(y_test)

In [24]:
y_train_encoded.shape

(120,)

In [25]:
y_test_encoded.shape

(30,)

In [27]:
# Defining the network architecture and activation functions
architecture = [4, 8, 3]  # Network architecture
activations = ['relu', 'softmax']  # Activation functions

# Use the build_network function to create the network
network = build_network(architecture, activations)

# Get the dimensionality of the network weights vector
dimensions = network.total_weights()

# Call pso_optimization function with the created network and parameters
best_weights = pso_optimization(
    num_particles=100,
    num_iterations=300,
    loss_func=sparse_categorical_crossentropy,
    network=network,
    train_data=X_train_scaled,
    train_labels=y_train_encoded,
    dimensions=dimensions)

# Set the best weights to the network
network.set_weights(best_weights)

# Test the network
output = network.forward(X_test_scaled)
predictions = np.argmax(output, axis=1)

# Print the results
print(classification_report(y_test_encoded, predictions))

Iteration 25/300, Current Best Loss: 0.06613688931788107
Iteration 50/300, Current Best Loss: 0.054865269993960006
Iteration 75/300, Current Best Loss: 0.050630708632469185
Iteration 100/300, Current Best Loss: 0.04602614064094665
Iteration 125/300, Current Best Loss: 0.04472557729102606
Iteration 150/300, Current Best Loss: 0.04455230425176431
Iteration 175/300, Current Best Loss: 0.043673787013478696
Iteration 200/300, Current Best Loss: 0.043031853485087326
Iteration 225/300, Current Best Loss: 0.042772767810788676
Iteration 250/300, Current Best Loss: 0.042715635432985274
Iteration 275/300, Current Best Loss: 0.04270318795662882
Iteration 300/300, Current Best Loss: 0.04263916643699015
Final Best Loss: 0.04263916643699015 on Iteration: 294
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                